<a href="https://colab.research.google.com/github/kunai-3txk/Compe_tonyobyo/blob/main/AMP_fit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [2]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 12.8 MB/s eta 0:00:00


In [23]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling
import os
import xgboost as xgb
import seaborn as sns; sns.set()

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_squared_error


import lightgbm as lgb #LightGBM
from lightgbm import LGBMRegressor

from catboost import CatBoostRegressor,Pool

# Configuration

In [18]:
class config:
    METRIC = 'RMSE' #RMSE or SMAPE
    RANDOM_STATE=100  
    FRAC = 0.2
    N_FOLD = 5
    
class paths:
    # kaggle環境ならTrue
    if 'KAGGLE_URL_BASE' in set(os.environ.keys()):
        common_path = "/kaggle/input/amp-parkinsons-disease-progression-prediction"
    
    # colaboratory環境ならTrue
    if 'COLAB_GPU' in set(os.environ.keys()):
        common_path = "/content/drive/MyDrive/AMP"
    
    TRAIN = common_path+"/train_all.csv"


# Function Definition

In [5]:
def display_feature_importance(feature_list,model):

    # 特徴量重要度を保管する dataframe を用意
    feature_importance = pd.DataFrame()
    feature_importance['feature'] = feature_list
    feature_importance['importance'] = model.feature_importances_

    order = list(feature_importance.groupby("feature")["importance"].mean().sort_values(ascending=False).index)[:30]
    # 可視化
    plt.figure(figsize=(10, 10))
    sns.barplot(x="importance",y="feature",data=feature_importance,order=order)
    plt.title('importance')
    plt.tight_layout()
    plt.show()

In [15]:
def score_cal(y_true, y_pred):
  if config.METRIC == 'SMAPE':
    smap = np.zeros(len(y_true))
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    ret = 100 * np.mean(smap)
  if config.METRIC == 'RMSE':
    ret = np.sqrt(mean_squared_error(y_true,y_pred))

  return ret

In [56]:
def fit_catboost_f1(X,y):

  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=config.FRAC, random_state=config.RANDOM_STATE,shuffle=True)

  train_pool = Pool(X_train, y_train)
  model = CatBoostRegressor(random_seed=config.RANDOM_STATE,logging_level='Silent',eval_metric = config.METRIC)
  model.fit(train_pool,plot=False, verbose=False)  

  test_score = score_cal(y_val, model.predict(X_val) )

  print(f"Test data average score : {test_score:.4f}"," seed ", config.RANDOM_STATE)

  return model

# Load Data

In [65]:
df_train = pd.read_csv(paths.TRAIN)
df_train.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,NPX_mean_by_id,NPX_std_by_id,NPX_median_by_id,NPX_mean_by_Uni,NPX_std_by_Uni,NPX_median_by_Uni,Abu_mean_by_id,Abu_std_by_id,Abu_median_by_id,Abu_mean_by_Pep,Abu_std_by_Pep,Abu_median_by_Pep
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,3.180508e+06,2.672700e+07,123757.0,2.596938e+06,702016.931653,2.534106e+06,748153.907014,4.231806e+06,93134.8,619334.666897,202617.423261,597764.341992
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN,2.942039e+06,2.391469e+07,129048.0,2.608845e+06,705203.512224,2.545728e+06,685218.599872,3.491859e+06,84814.9,620241.001591,203152.040437,598682.013953
3,55_9,55,9,8.0,9.0,30.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55_12,55,12,10.0,10.0,41.0,0.0,1.0,3.145608e+06,2.612674e+07,124498.0,2.562630e+06,692742.468271,2.500628e+06,740535.423563,3.882536e+06,91575.2,612046.308868,200234.524653,590729.986479


In [66]:
#pivot
df_train_pv=df_train[['visit_id','patient_id','visit_month','updrs_1','updrs_2','updrs_3','updrs_4']].dropna(axis=0).pivot(index="patient_id",columns='visit_month',values='visit_id').reset_index()
df_train_pv.head()

#testデータ用の候補探し
test_patient_candidate_list = list(df_train_pv[
    ~df_train_pv[0].isna() & ~df_train_pv[6].isna() & ~df_train_pv[12].isna() & ~df_train_pv[24].isna()
]['patient_id'].values)
len(test_patient_candidate_list)


56

In [92]:
test_patient_list = [1517,3863]
test_month_list = [0,6,12,24] 
#test_month_list = [0,3,6,9,12,18,24]
df_test = df_train[df_train['patient_id'].isin(test_patient_list) & df_train['visit_month'].isin(test_month_list)]


df_test_vs = pd.DataFrame()

for i in range(0,3):
  df_test_vs_ = pd.DataFrame()
  df_test_vs_['id']=df_test[['visit_id']] + '_updrs_{0}'.format(i+1)
  df_test_vs_['updrs']=df_test[['updrs_{0}'.format(i+1)]]
  df_test_vs = pd.concat([df_test_vs,df_test_vs_],axis=0)



,id,updrs
28,1517_0_updrs_1,11.0
29,1517_6_updrs_1,17.0
30,1517_12_updrs_1,20.0
32,1517_24_updrs_1,19.0
65,3863_0_updrs_1,8.0
66,3863_6_updrs_1,9.0
67,3863_12_updrs_1,11.0
69,3863_24_updrs_1,11.0
28,1517_0_updrs_2,6.0
29,1517_6_updrs_2,4.0


# Fit

In [129]:

predict_order_list = [1,2,3]

# Stage1 nothing => updrs_1
df_train_ = df_train.drop(['visit_id','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication'],axis=1)

for order in predict_order_list:
  train_X = df_train_.values
  train_y = df_train['updrs_{0}'.format(order)].values
  model = fit_catboost_f1(train_X,train_y)
  df_train_['updrs_{0}'.format(order)] = model.predict(train_X)

  if order == 1:
        model_1 = model
  if order == 2:
        model_2 = model
  if order == 3:
        model_3 = model

df_test_ = df_test.drop(['visit_id','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication'],axis=1)

for order in predict_order_list:
  test_X = df_test_.values
  test_y = df_test['updrs_{0}'.format(order)].values
  if order == 1:
        model_ = model_1
  if order == 2:
        model_ = model_2
  if order == 3:
        model_ = model_3
  df_test_['updrs_{0}'.format(order)] = model_.predict(test_X)


df_test_vs_predict = pd.DataFrame()
for i in range(0,3):
  df_test_vs_predict_ = pd.DataFrame()
  df_test_vs_predict_['id'] = df_test_['patient_id'].astype(str) + '_' + df_test_['visit_month'].astype(str) + '_updrs_{0}'.format(i+1)
  df_test_vs_predict_['updrs'] = df_test_[['updrs_{0}'.format(i+1)]]
  df_test_vs_predict = pd.concat([df_test_vs_predict,df_test_vs_predict_],axis=0)

print('Score Predict',score_cal(df_test_vs['updrs'],df_test_vs_predict['updrs']))

#X = df_train.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1).values
#y = df_train['updrs_{0}'.format(i + 1)].values
#train_column_list = df_train_.columns

Test data average score : 3.9578  seed  100
Test data average score : 4.4958  seed  100
Test data average score : 10.4876  seed  100
Score Predict 6.211127439877866


In [ ]:

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=config.FRAC, random_state=config.RANDOM_STATE,shuffle=True)

# df_log = pd.DataFrame() #Log用DFの定義
# models = []                          # 各 fold のモデル
# scores = 0.0   
# best_score = np.inf
# preds = np.zeros(len(X_test))

# gf = GroupShuffleSplit(n_splits=config.N_FOLD,test_size=0.2,random_state=config.RANDOM_STATE)
# groups = X_train[:,0]

# for fold, (trn_idx, val_idx) in enumerate(gf.split(X_train,y_train,groups)):

#     X_trn, y_trn = X_train[trn_idx,:], y_train[trn_idx]
#     X_val, y_val = X_train[val_idx,:], y_train[val_idx]

#     train_pool = Pool(X_trn, y_trn)
#     validate_pool = Pool(X_val, y_val)

#     model = CatBoostRegressor(random_seed=config.RANDOM_STATE,logging_level='Silent',eval_metric = config.METRIC)
#     model.fit(train_pool, eval_set = validate_pool, early_stopping_rounds = 10, use_best_model=True, plot=False, verbose=False)  
    
#     score = score_cal(y_val,model.predict(X_val))
#     scores += score / config.N_FOLD
#     models.append(model)

#     if score < best_score:
#         best_score = score
#         best_model = model
#         best_fold = fold+1
#     print("Fold :" , fold+1)

# print(f"fold average score : {scores:.4f}")
# print(f"fold best score : {best_score:.4f}"," FOLD ",best_fold)
# #****************************
# # test データ

# # 各モデルで推論
# for model in models:
#     preds += model.predict(X_test) / len(models) 
# test_score = score_cal(y_test, preds)
# pred_best = best_model.predict(X_test)

# print(f"Test data average score : {test_score:.4f}"," seed ", config.RANDOM_STATE)
# print(f"Test data best socre : {score_cal(y_test, pred_best):.4f}"," seed ", config.RANDOM_STATE)
# print("***********************")
# #display_feature_importance(train_column_list,best_model)
# feature_importance = pd.DataFrame()
# feature_importance['feature'] = train_column_list
# feature_importance['importance'] = best_model.feature_importances_

# feature_importance.groupby("feature")["importance"].mean().sort_values(ascending=False)[:30]